# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [4]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,file_url
0,02-04-21,Pembangunan Aplikasi Mobile Pengenalan Objek U...,Jurnal Teknologi Informasi dan Ilmu Komputer (...,Mobile phone usage has been very close to earl...,"Supriadi, M., Rachmawati, E., & Arifianto, A. ...",pembangunan-aplikasi-mobile-pengenalan-objek-u...,https://jtiik.ub.ac.id/index.php/jtiik/article...,https://jtiik.ub.ac.id/index.php/jtiik/article...
1,16-10-20,Lip Motion Recognition for Indonesian Vowel Ph...,2020 3rd International Conference on Computer ...,Lip motion recognition is a technique for inte...,"Maxalmina, S. Kahfi, K. N. Ramadhani and A. Ar...",lip-motion-recognition-for-indonesian-vowel-ph...,https://ieeexplore.ieee.org/document/9274562,https://drive.google.com/file/d/1FzAEEjs6WVtxy...
2,02-10-20,Classifying Skin Cancer in Digital Images Usin...,"Indonesia Journal on Computing (Indo-JC), Vol....",Skin cancer is a hazardous disease that can in...,"Aliyah, Z., Arifianto, A., & Sthevanie, F. (20...",classifying-skin-cancer-in-digital-images-usin...,https://socj.telkomuniversity.ac.id/ojs/index....,https://socj.telkomuniversity.ac.id/ojs/index....
3,02-10-20,Pneumonia Classification from X-ray Images Usi...,"Indonesia Journal on Computing (Indo-JC), Vol....","Pneumonia is a virus, bacterium, and fungi inf...","Ahnafi, A. H., Arifianto, A., & Ramadhani, K. ...",pneumonia-classification-from-x-ray-images-usi...,https://socj.telkomuniversity.ac.id/ojs/index....,https://socj.telkomuniversity.ac.id/ojs/index....
4,06-08-20,Recognizing Soft Biometric on Pedestrian Using...,2020 International Conference on Data Science ...,Video surveillance is very important in automa...,"T. B. Siswoyo, A. Arifianto and K. N. Ramadhan...",recognizing-soft-biometric-on-pedestrian-using...,https://ieeexplore.ieee.org/document/9213066,https://drive.google.com/file/d/1U0BjPG6RBVXBy...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [5]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [10]:
import os
for row, item in publications.iterrows():
    print('-------------')
    print(row, item)
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.file_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

-------------
0 pub_date                                              02-04-21
title        Pembangunan Aplikasi Mobile Pengenalan Objek U...
venue        Jurnal Teknologi Informasi dan Ilmu Komputer (...
excerpt      Mobile phone usage has been very close to earl...
citation     Supriadi, M., Rachmawati, E., & Arifianto, A. ...
url_slug     pembangunan-aplikasi-mobile-pengenalan-objek-u...
paper_url    https://jtiik.ub.ac.id/index.php/jtiik/article...
file_url     https://jtiik.ub.ac.id/index.php/jtiik/article...
Name: 0, dtype: object
-------------
1 pub_date                                              16-10-20
title        Lip Motion Recognition for Indonesian Vowel Ph...
venue        2020 3rd International Conference on Computer ...
excerpt      Lip motion recognition is a technique for inte...
citation     Maxalmina, S. Kahfi, K. N. Ramadhani and A. Ar...
url_slug     lip-motion-recognition-for-indonesian-vowel-ph...
paper_url         https://ieeexplore.ieee.org/document/9274562


UnicodeEncodeError: 'charmap' codec can't encode character '\ufffd' in position 972: character maps to <undefined>

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md  2015-10-01-paper-title-number-3.md
2010-10-01-paper-title-number-2.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).